# tiff_file_tools Function Usage

This notebook contains examples for how the simple functions in [tiff_file_tools.py](https://github.com/MIT-AI-Accelerator/multiearth-challenge/blob/main/src/multiearth_challenge/tiff_file_tools.py) can be used to extract metadata from and retrieve aligned imagery from TIFF files provided as part of the MultiEarth challenge. These TIFF files serve as an alternate data storage format for those who prefer them to NETCDF files.

In [1]:
import pkg_resources

from matplotlib import pyplot as plt

from multiearth_challenge import tiff_file_tools as tft

%matplotlib inline
%load_ext autoreload
%autoreload 2

### Filename Parsing
The MultiEarth 2023 TIFF files have encoded into their filename the collecting sensor's name, the sensor band, the latitude and longitude position of the image, and the collection date.  The function [parse_filename_parts](https://github.com/MIT-AI-Accelerator/multiearth-challenge/blob/c2318665ab94451eea4bd1b9e31a71655c6be001/src/multiearth_challenge/zip_file_tools.py#L12) will extract this metadata from the filename.

In [2]:
tiff_path = pkg_resources.resource_filename("multiearth_challenge", "data/sample_dataset/sent1/Sentinel1_VH_-55.20_-4.39_2019_06_08.tiff")
file_info = tft.parse_filename_parts(tiff_path)
print(file_info)

{'sensor': 'Sentinel1', 'band': 'VH', 'lat': -4.39, 'lon': -55.2, 'date': datetime.date(2019, 6, 8)}


### Aligned Imagery
The MultiEarth sub-challenges involve retrieving spatially and temporally aligned imagery. The function [get_aligned_images](https://github.com/MIT-AI-Accelerator/multiearth-challenge/blob/c2318665ab94451eea4bd1b9e31a71655c6be001/src/multiearth_challenge/zip_file_tools.py#L79) will identify images that are at the same location and collected within a specified time window of a target image.

In [3]:
# The paths for images to find spatially and temporally aligned images for.
target_images = [
    pkg_resources.resource_filename("multiearth_challenge", "data/sample_dataset/sent2/Sentinel2_B1_-55.20_-4.39_2019_06_06.tiff"), 
    pkg_resources.resource_filename("multiearth_challenge", "data/sample_dataset/sent2/Sentinel2_B1_-55.20_-4.39_2019_06_11.tiff"), 
    pkg_resources.resource_filename("multiearth_challenge", "data/sample_dataset/sent2/Sentinel2_B2_-55.20_-4.39_2019_07_01.tiff"), 
    pkg_resources.resource_filename("multiearth_challenge", "data/sample_dataset/sent2/Sentinel2_B1_-55.20_-4.39_2020_07_20.tiff"), 
]

#  The paths for images to search to find which ones are aligned with images1.
source_images = [
    pkg_resources.resource_filename("multiearth_challenge", "data/sample_dataset/sent1/Sentinel1_VH_-55.20_-4.39_2019_06_08.tiff"),
    pkg_resources.resource_filename("multiearth_challenge", "data/sample_dataset/sent1/Sentinel1_VH_-55.20_-4.39_2019_05_27.tiff"),
    pkg_resources.resource_filename("multiearth_challenge", "data/sample_dataset/sent1/Sentinel1_VH_-55.20_-4.39_2019_07_02.tiff"),
]

# A window in +- days (inclusive) specifying whether two images were collected close enough in time to be considered aligned.
date_window = (-10, 10)

image_alignment = tft.get_aligned_images(
        target_images,
        source_images,
        date_window,
)

The output is a dictionary with keys being each target image path and the associated value being a list of aligned source image paths.

In [4]:
for target_path, source_paths in image_alignment.items():
    print(f"Target image: {target_path.name} aligned with:")
    for source_path in source_paths:
        print(f"    Source image: {source_path.name}")
    if not len(source_paths):
        print("    None")

Target image: Sentinel2_B1_-55.20_-4.39_2019_06_06.tiff aligned with:
    Source image: Sentinel1_VH_-55.20_-4.39_2019_06_08.tiff
    Source image: Sentinel1_VH_-55.20_-4.39_2019_05_27.tiff
Target image: Sentinel2_B1_-55.20_-4.39_2019_06_11.tiff aligned with:
    Source image: Sentinel1_VH_-55.20_-4.39_2019_06_08.tiff
Target image: Sentinel2_B2_-55.20_-4.39_2019_07_01.tiff aligned with:
    Source image: Sentinel1_VH_-55.20_-4.39_2019_07_02.tiff
Target image: Sentinel2_B1_-55.20_-4.39_2020_07_20.tiff aligned with:
    None
